In [26]:
import pandas as pd
import ast
import gensim
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import strip_punctuation
print((gensim.__version__))  # needed 3.8.3   -> pip install gensim==3.8.3
import pickle
import numpy as np

4.1.2


In [7]:
y_test = pd.read_csv("C:/Users/berl03/Desktop/LIME/1 příprava dat/y_test.csv")
print(len(y_test))
print(y_test.head(2))

8645
   Unnamed: 0                        doi  target  \
0           0  10.1186/s13756-019-0662-8       0   
1           1  10.1038/s41541-019-0148-y       0   

                                    abstract_cleaned  \
0  antimicrobial resistance amr compromise treatm...   
1  mycobacterium tuberculosis mtb responsible dea...   

                                  abstract_tokenized  
0  ['antimicrobial', 'resistance', 'amr', 'compro...  
1  ['mycobacterium', 'tuberculosis', 'mtb', 'resp...  


In [27]:
model_w2v=gensim.models.Word2Vec.load("C:/Users/berl03/Desktop/LIME/1 příprava dat/w2v_published_only_english_between_2019 and 2020.model")
model_lr = lreg_w2v_avg = pickle.load(open("C:/Users/berl03/Desktop/LIME/1 příprava dat/lreg_w2v_avg_2020.sav", 'rb'))

In [13]:
y_test['abstract_tokenized'] = y_test['abstract_tokenized'].apply(ast.literal_eval)
words_list_with_target = list(set([a for b in y_test['abstract_tokenized'] for a in b]))
print(len(words_list_with_target))

34509


In [22]:
#### words of word2vec model dictionary trained on whole kaggle corpus
words = model_w2v.wv.key_to_index.keys()
we_dict = {word:model_w2v.wv[word] for word in words}
words_list_total = pd.DataFrame(we_dict.items())
print(len(words_list_total))

82402


In [23]:
def score_of_word(model_w2v,lreg_w2v):
    
    words = model_w2v.wv.key_to_index.keys()
    we_dict = {word:model_w2v.wv[word] for word in words}
    df_of_words_vectors = pd.DataFrame(we_dict.items())
    df_of_words_vectors_ar = np.array(df_of_words_vectors[1].to_list())

    probs = lreg_w2v.predict_proba(df_of_words_vectors_ar)

    score_of_word = pd.DataFrame(list(probs[:,1]),columns =["score"] , index=df_of_words_vectors[0])
    score_of_word = score_of_word.sort_values("score",ascending=False).reset_index()
    score_of_word.columns = ["word","score"]
    
    return score_of_word

In [30]:
score_of_word_lr_avg_2 =  score_of_word(model_w2v,model_lr).sort_values(by = ["score"], ascending=False)

In [31]:
score_of_word_lr_avg_2

word     score
0           hubei  0.999945
1       2019‐ncov  0.999918
2           wuhan  0.999908
3       sars-like  0.999695
4         hamster  0.999681
...           ...       ...
82397     failing  0.000080
82398          ho  0.000071
82399  cellulitis  0.000063
82400         aco  0.000058
82401         icd  0.000050

[82402 rows x 2 columns]

In [32]:
score_of_word_lr_avg_2.to_csv("SMER_new.csv")